In [6]:
import torch
from torch.utils.data import DataLoader

from models.linear_decoder import LinearDecoder
from bravo_datasets import CityscapesDataset
from inference import normal_eval
from clean_dataset import visualize_batch

In [7]:
encoder_name = ['vit_large_patch14_dinov2', "eva02_large_patch14_224", "depth-anything/Depth-Anything-V2-Base-hf"]
img_size = (1024, 1024) 
num_classes = 19
device = torch.device('cuda:0')

In [ ]:
out_of_context = '/media/data/workspace_Giuseppe/code/datasets/BRAVO/outofcontext'
synrain = '/media/data/workspace_Giuseppe/code/datasets/BRAVO/synrain'
ignore_idx_out_of_context = [0,1,9]

dataset = CityscapesDataset(root_dir=out_of_context, ignore_idx=ignore_idx_out_of_context)
eval_dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=8)

In [8]:
model_name = 'eva02_large_patch14_224.mim_m38m'
model = LinearDecoder(
    model_name,
    num_classes=19,
    img_size=img_size,
    pretrained=True,
    sub_norm=False,
    patch_size=8,
)

/home/artemis/miniconda3/envs/benchmark-vfm-ss/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [ ]:
patch_size = 16
checkpoint_path_dinov2large = '/media/data/workspace_Giuseppe/code/checkpoints/CityScapes/dinov2largepatch8linear/best-checkpoint-epoch=41-val_0_miou=0.85.ckpt'
checkpoint_path_depthv2base = '/media/data/workspace_Giuseppe/code/checkpoints/CityScapes/depthv2base_16psize/best-checkpoint-epoch=36-val_0_miou=0.81.ckpt'
model = LinearDecoder(
    encoder_name[2],
    num_classes = num_classes,
    img_size = img_size,
    sub_norm = False, 
    patch_size = patch_size,
    pretrained = True
)

model.to(device)
model.eval()

checkpoint = torch.load(checkpoint_path_depthv2base)
  
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
normal_eval(model=model, 
          img_size=img_size, 
          device=device, 
          num_classes=num_classes, 
          eval_dataloader=eval_dataloader)

In [1]:
from bravo_datasets import ACDCDataset

acdc_root = '/media/data/workspace_Giuseppe/code/datasets/BRAVO/ACDC'
acdc_dataset = ACDCDataset(root_dir=acdc_root, split='train', img_size=1024)

/home/artemis/miniconda3/envs/benchmark-vfm-ss/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----

python3 inference.py \
    --model.network.encoder_name eva02_large_patch14_224.mim_m38m \
    --model.network.patch_size 8 \
    --checkpoint_path /home/kronos/workspace_Giuseppe/checkpoints_CS/eva02large_patch8/best-checkpoint-epoch=39-val_0_miou=0.82.ckpt \
    --img_size 1024 \
    --dataset ACDC \
    --root /home/kronos/workspace_Giuseppe/data/ACDC \
    --batch_size 32 \
    --num_workers 8

python3 inference.py \
    --model.network.encoder_name vit_large_patch14_dinov2 \
    --model.network.patch_size 8 \
    --checkpoint_path /media/data/workspace_Giuseppe/code/checkpoints/CityScapes/dinov2largepatch8linear/best-checkpoint-epoch=41-val_0_miou=0.85.ckpt \
    --img_size 1024 \
    --dataset out_of_context \
    --root /media/data/workspace_Giuseppe/code/datasets/BRAVO/outofcontext \
    --batch_size 4 \
    --num_workers 8

python3 inference.py \
    --model.network.encoder_name depth-anything/Depth-Anything-V2-Base-hf \
    --model.network.patch_size 16 \
    --checkpoint_path /media/data/workspace_Giuseppe/code/checkpoints/CityScapes/depthv2base_16psize/best-checkpoint-epoch=36-val_0_miou=0.81.ckpt \
    --img_size 1024 \
    --dataset synrain \
    --root /media/data/workspace_Giuseppe/code/datasets/BRAVO/synrain \
    --batch_size 4 \
    --num_workers 8